## Preguntas: Falta agregarle un ejemplo de cada item
1- La idea es que despache de a paquetes? osea si tengo 10 galletas, no las parto en 2 sino que mando los 10 enteros?

2- Como armo el recorrido? pobmunedo + dia? osea un camion puede ir a una localidad, no intra localidad?

3- Que pasa si tengo un pedido que requiere multiples camiones? como se manjea la disponibilidad?

Ejemplo: folio	C23ORI06 que entró el día 2023-03-13, tiene un volumen total de 243.52 y peso 55619.51. Entraron 24 pedidos

4- Que pasa si mandan un folio que no llega a llenar el camion y pasan varios días hasta que llega un nuevo folio? se fuerza el envio con el camion a medio armar o se lo deja al operario que decida?

5- Muchas veces cuando lo miramos a nivel poblacion no converge porque hay muchos pedidos enormes. por ejemplo: GALLETAS MARIA MARIBEL 20/160 GR	cantidad:200	volumen:719.8000	peso:3.2312	precio:29214.0000
estos pedidos hacen que no se pueda tener granularidad en la resolucion


### Pendientes:

En el excel de tarifas hay precios disintos para mismo vehiculo. Que hacemos?

	POBLACION	TIPO_VEHICULO	TARIFA
372	GUADALAJARA, JAL	CAMIONETA	1200
373	GUADALAJARA, JAL	CAMIONETA	1500

### Hay que pensar como guardar la metadata de los vehiculos y los tarifarios

### Estas poblaciones no tiene tarifario

pobmunedo	volumen	peso	total
12	BAHIA DE BANDERAS, NAY	18.5146	4294.432	124067.7540
14	CALERA DE VICTOR ROSALES, ZAC	1.3829	720.275	55612.8000
28	COQUIMATLAN, COL	7.9766	2061.318	46173.9636
31	CUATITLAN DE GARCIA BARRAGAN, JAL	22.8274	4656.678	154876.3498
43	GOMEZ PALACIO, DGO	2.9875	635.300	19.2070
46	HOSTOTIPAQUILLO, JAL	24.0646	5278.958	207569.7792
65	MISANTLA, VER	7.3740	1462.490	259.2076
68	NAOLINCO, VER	42.1080	8397.052	154516.4964
84	PUEBLO NUEVO,DGO	26.1807	5404.247	116990.2673
105	TESISTAN, JAL	19.3220	4458.020	99492.3851
125	VILLA DE ARISTA, SLP	3.6630	542.520	3.0000
129	XICO, VER	52.2520	10506.410	172104.0069

In [ ]:
# Pasos que tengo que trabajar:
Hacerlo mono mochila. Primero busco la mochila mas grande que podría caber
Despues itento llenarla
Despues saco los que ya resolvi y paso a la proxima mochila (siempre evaluo cual es la mochila mas grande que sigue y aplico funcion de corte)
Tengo que armar funcion de costo para pesar mas a los items que quedaron viejos

In [2]:
import pandas as pd

import ortools
from ortools.linear_solver import pywraplp

In [3]:
df = pd.read_excel("info.xlsx",sheet_name="detalle")

df["fecha"] = pd.to_datetime(df["fecha"],infer_datetime_format=True)
df["dia"] = pd.to_datetime(df["fecha"].dt.date,infer_datetime_format=True)
df["pobmunedo"] = df["pobmunedo"].str.strip()

resultados = pd.DataFrame([])
to_combine = []
resagados_append = []

fechas = [pd.to_datetime(x).strftime("%Y-%m-%d") for x in df.dia.unique()]

In [36]:
important_cols = ['folio','pobmunedo', 'cod_prod','cantidad_autorizada','peso', 'volumen', 'total', 'dia']
to_process = df.loc[(df.dia >= "2023-02-03")&(df.dia <= "2023-03-10") & (df.pobmunedo == "TORREON, COAH")][important_cols]

data = {}
values = to_process.shape[0]*[1]
weights = to_process.peso.to_list()
volume = to_process.volumen.to_list()
price = to_process.total.to_list()

data['values'] = values
data['weights'] = weights
data['volume'] = volume
data['price'] = price
data['items'] = list(range(len(weights)))
data['num_items'] = len(values)

#number_bags = 4 #All have the same capacity of 50 pounds
#data['bag_weight'] = [5000, 15000, 25000, 50000] #pounds
#data['bag_volume'] = [21,40,84,130] #
#data['bag_price'] = [6500,9200,11000,22000]
##I wanted to change the values at a later data
#data['bags'] = list(range(number_bags))

number_bags = 1 #All have the same capacity of 50 pounds
data['bag_weight'] = [15000] #pounds
data['bag_volume'] = [40] #
data['bag_price'] = [9200]
#I wanted to change the values at a later data
data['bags'] = list(range(number_bags))

solver = pywraplp.Solver.CreateSolver('SCIP')

#Prepare solver matrix
x = {}
for i in data['items']:
    for j in data['bags']:
        x[(i,j)] = solver.IntVar(0,1,'x_%i_%i' % (i, j))
        
#Constraint for an item being placed in 1 knapsack
for i in data['items']:
    solver.Add(sum(x[i,j] for j in data['bags'])<=1)

#Knapsack max Constraint
for j in data['bags']:
    solver.Add(sum(x[(i,j)]*data['weights'][i] for i in data['items']) <= data['bag_weight'][j])
    solver.Add(sum(x[(i,j)]*data['volume'][i] for i in data['items']) <= data['bag_volume'][j])
    solver.Add(sum(x[(i,j)]*data['price'][i] for i in data['items'])*0.1 >= data['bag_price'][j])
    
#Knapsack min Constraint
for j in data['bags']:
    solver.Add(
        (sum(x[(i,j)]*data['volume'][i] for i in data['items']) >= data['bag_volume'][j]*0.8) 
        or
        sum(x[(i,j)]*data['weights'][i] for i in data['items']) >= (data['bag_weight'][j]*0.8)
    )

# Maximize value, I can manage delay
objective = solver.Objective()
for i in data['items']:
    for j in data['bags']:
        objective.SetCoefficient(x[(i,j)], data['values'][i])
objective.SetMaximization()

In [37]:
solv = solver.Solve()
print(solv)
l_ = [[]]

if solv == pywraplp.Solver.OPTIMAL:
    for j in data['bags']:
        bag_value = 0
        bag_weight = 0
        bag_volume= 0
        bag_rad = 0
        print('\n','Bag', j+1 , '\n')
        for i in data['items']:
            if x[i,j].solution_value()>0:
                print('Item:', i , 
                      'value',data['values'][i],
                      'Weight', data['weights'][i], 
                      'Volume',data['volume'][i],
                      'Radiation', data['price'][i]
                     )
                l_[j] += [i]
                bag_value += data['values'][i]
                bag_weight += data['weights'][i]
                bag_volume += data['volume'][i]
                bag_rad += data['price'][i]
        print('Packed Knapsack Value: ', bag_value)
        print('Packed Knapsack Weight: ', bag_weight)
        print('Packed Knapsack Volume: ', bag_volume)
        print('Pack Knapsack Radiation: ', bag_rad)

0

 Bag 1 

Item: 2 value 1 Weight 381.3 Volume 1.89 Radiation 3.0
Item: 4 value 1 Weight 97.5 Volume 0.355 Radiation 54.0
Item: 5 value 1 Weight 156.1 Volume 0.75 Radiation 54.0
Item: 6 value 1 Weight 52.38 Volume 0.225 Radiation 16.2
Item: 7 value 1 Weight 43.33 Volume 0.127 Radiation 0.108
Item: 8 value 1 Weight 43.33 Volume 0.127 Radiation 10.8
Item: 9 value 1 Weight 25.02 Volume 0.036 Radiation 757.68
Item: 12 value 1 Weight 165.0 Volume 0.3685 Radiation 17772.48
Item: 13 value 1 Weight 25.32 Volume 0.0486 Radiation 757.68
Item: 14 value 1 Weight 354.256 Volume 1.2596 Radiation 30924.63
Item: 15 value 1 Weight 98.5523 Volume 0.9817 Radiation 21847.98
Item: 16 value 1 Weight 388.448 Volume 0.7001 Radiation 19825.79
Item: 17 value 1 Weight 877.834 Volume 1.611 Radiation 28087.23
Item: 18 value 1 Weight 207.7 Volume 0.508 Radiation 38837.09
Item: 19 value 1 Weight 129.6 Volume 0.1573 Radiation 6589.73
Item: 20 value 1 Weight 1074.364 Volume 2.1289 Radiation 34375.42
Item: 21 value 1 

In [220]:
def sizer(n,div):
    spliter = int(n/div)
    
    
def expand(d):
    r = []
    
    if((d["cantidad_autorizada"] >= 10) & d["cantidad_autorizada"] < 100):
        return []
    return [d]

l_ = []

#for ind,line in to_process.iterrows():
#    l_ += expand(line.to_dict())
    


[{'folio': 'PBP39614  ',
  'pobmunedo': 'TORREON, COAH',
  'cod_prod': 3801,
  'cantidad_autorizada': 50,
  'peso': 252.2,
  'volumen': 1.3,
  'total': 54.0,
  'dia': Timestamp('2023-02-04 00:00:00')},
 {'folio': 'PBP39614  ',
  'pobmunedo': 'TORREON, COAH',
  'cod_prod': 4914,
  'cantidad_autorizada': 400,
  'peso': 1439.6,
  'volumen': 6.48,
  'total': 0.0432,
  'dia': Timestamp('2023-02-04 00:00:00')},
 {'folio': 'PBP39614  ',
  'pobmunedo': 'TORREON, COAH',
  'cod_prod': 4672,
  'cantidad_autorizada': 29,
  'peso': 381.3,
  'volumen': 1.89,
  'total': 3.0,
  'dia': Timestamp('2023-02-04 00:00:00')},
 {'folio': 'PBP39614  ',
  'pobmunedo': 'TORREON, COAH',
  'cod_prod': 3869,
  'cantidad_autorizada': 300,
  'peso': 1610.7,
  'volumen': 7.89,
  'total': 3.0,
  'dia': Timestamp('2023-02-04 00:00:00')},
 {'folio': 'PBP39614  ',
  'pobmunedo': 'TORREON, COAH',
  'cod_prod': 4894,
  'cantidad_autorizada': 50,
  'peso': 97.5,
  'volumen': 0.355,
  'total': 54.0,
  'dia': Timestamp('2023-0

In [149]:
df.loc[(df.dia >= "2023-02-03")&(df.dia <= "2023-02-10")].groupby("pobmunedo").agg({"volumen":"sum","peso":["sum","size"]})

volumen        peso     
                                 sum         sum size
pobmunedo                                            
CANDELARIA, CAMP             26.4354   3998.7250   11
CD. GUZMAN, JAL              15.4570   3249.2700    6
CD. VICTORIA, TMPS           28.9724   7214.6550    8
CHAMPOTÓN, CAMP               1.1466    575.4250    4
CHETUMAL, Q.R.               44.2289  10746.2965   28
CHIHUITAN, OAX               61.6103  22394.3865   22
COATEPEC, VER                 4.6800    907.9200    1
COLIMA, COL                  50.2386  11559.4740   13
CORTAZAR, GTO                19.4400   3779.6000    2
CULIACAN, SIN                 6.1603   1242.1150    6
DURANGO, DGO                 18.2481   3701.1290    9
GUADALAJARA, JAL             64.8280  13557.8050   33
HERMOSILLO, SON               6.7942   2354.1150   13
LAZARO CÁRDENAS, Q.R.        52.1347   8915.0460   24
LEON, GTO                    26.8711   5198.7750    7
LOS MOCHIS, SIN              28.9500   5956.2200    6
LOS REYES, MICH              10.9968   2229.5420    2
MEXICALI, BC                 26.4613   6176.0040   21
MISANTLA, VER                 7.3740   1462.4900    2
MORELIA, MICH                19.4584   4163.9850   20
PALOMARES, OAX               94.2600  18698.0400    3
PATZCUARO, MICH               3.6750    795.8140    5
SAN MIGUEL DE ALLENDE, GTO    7.4918   1469.2790    6
TEHUACAN, PUE                10.8990   2398.3160    4
TEPIC, NAY                  155.9501  30806.5710   13
TIHUATLAN, VER               36.2041  11693.7850   16
TLAQUEPAQUE, JAL              8.8429   1814.6790    6
TOMATLAN, JAL                 3.7692    874.8700   10
TORREON, COAH                96.2505  23104.4815   38
TOTUTLA, VER                 22.7700   4531.4300    2
TUXTLA GUTIERREZ, CHIS      109.6115  21359.8970    6
XICO, VER                     2.9160    647.8200    1
ZAPOPAN, JAL                  5.4650   1180.3000    4

In [229]:
import numpy as np

def fix_splitter(n,div):
    
    splits = []
    remain = list(range(len(to_split)))

    while(len(remain) > 0):
        if(len(remain) > n):
            sampleo = remain[0:n]
            remain = [x for x in remain if x not in sampleo]
            splits.append(np.array(to_split)[sampleo].tolist())
        else:
            splits.append(np.array(to_split)[remain].tolist())
            break
            
    if(len(splits) > n):
        for i,x in enumerate(splits[-1]): # 
            splits[i] += [x]
    
        splits.pop()

    return splits

[len(x)for x in splitter(list(range(22)),5)]

[5, 5, 5, 5, 2]

In [243]:
to_process.head(40)

,folio,pobmunedo,cod_prod,cantidad_autorizada,peso,volumen,total,dia
2895,PBP39614,"TORREON, COAH",3801,50,252.2000,1.3000,54.0000,2023-02-04
2896,PBP39614,"TORREON, COAH",4914,400,1439.6000,6.4800,0.0432,2023-02-04
2897,PBP39614,"TORREON, COAH",4672,29,381.3000,1.8900,3.0000,2023-02-04
2898,PBP39614,"TORREON, COAH",3869,300,1610.7000,7.8900,3.0000,2023-02-04
2899,PBP39614,"TORREON, COAH",4894,50,97.5000,0.3550,54.0000,2023-02-04
2900,PBP39614,"TORREON, COAH",3702,50,156.1000,0.7500,54.0000,2023-02-04
2901,PBP39614,"TORREON, COAH",3710,15,52.3800,0.2250,16.2000,2023-02-04
2902,PBP39614,"TORREON, COAH",50457,10,43.3300,0.1270,0.1080,2023-02-04
2903,PBP39614,"TORREON, COAH",4932,10,43.3300,0.1270,10.8000,2023-02-04
3124,C22COA05,"TORREON, COAH",3922,6,25.0200,0.0360,757.6800,2023-02-09


In [98]:
0.8*5000

4000.0

In [ ]:
OPTIMAL = _pywraplp.Solver_OPTIMAL
r""" optimal."""
FEASIBLE = _pywraplp.Solver_FEASIBLE
r""" feasible, or stopped by limit."""
INFEASIBLE = _pywraplp.Solver_INFEASIBLE
r""" proven infeasible."""
UNBOUNDED = _pywraplp.Solver_UNBOUNDED
r""" proven unbounded."""
ABNORMAL = _pywraplp.Solver_ABNORMAL
r""" abnormal, i.e., error of some kind."""
MODEL_INVALID = _pywraplp.Solver_MODEL_INVALID
r""" the model is trivially invalid (NaN coefficients, etc)."""
NOT_SOLVED = _pywraplp.Solver_NOT_SOLVED

In [209]:
to_process = df.loc[(df.dia >= "2023-02-03")&(df.dia <= "2023-02-20") & (df.pobmunedo == "TORREON, COAH")]

data = {}
values = to_process.shape[0]*[1]
weights = to_process.peso.to_list()
volume = to_process.volumen.to_list()
price = to_process.total.to_list()

data['values'] = values
data['weights'] = weights
data['volume'] = volume
data['price'] = price
data['items'] = list(range(len(weights)))
data['num_items'] = len(values)

number_bags = 4 #All have the same capacity of 50 pounds
data['bag_weight'] = [5000, 15000, 25000, 50000] #pounds
data['bag_volume'] = [21,40,84,130] #
data['bag_price'] = [6500,9200,11000,22000]
#I wanted to change the values at a later data
data['bags'] = list(range(number_bags))

solver = pywraplp.Solver.CreateSolver('SCIP')

#Prepare solver matrix
x = {}
fraction = {}

for i in data['items']:
    for j in data['bags']:
        x[(i,j)] = solver.IntVar(0,1,'x_%i_%i' % (i, j))
        fraction[(i,j)] = solver.NumVar(0, 1, 'fraction_%i_%i' % (i, j))
        
#Constraint for an item being placed in 1 knapsack
for i in data['items']:
    solver.Add(sum(x[i,j] for j in data['bags'])<=1)

#Knapsack max Constraint
for j in data['bags']:
    solver.Add(sum(fraction[(i,j)]*data['weights'][i] for i in data['items']) <= data['bag_weight'][j])
    solver.Add(sum(fraction[(i,j)]*data['volume'][i] for i in data['items']) <= data['bag_volume'][j])
    #solver.Add(sum(fraction[(i,j)]*data['price'][i] for i in data['items'])*0.1 <= data['bag_price'][j])
    
#Knapsack min Constraint
for j in data['bags']:
    solver.Add(
        (sum(fraction[(i,j)]*data['volume'][i] for i in data['items']) >= (data['bag_volume'][j]*0.8)) 
        or
        sum(fraction[(i,j)]*data['weights'][i] for i in data['items']) >= (data['bag_weight'][j]*0.8)
    )

# Consistency constraint: fraction is only taken if the item is selected
for i in data['items']:
    for j in data['bags']:
        solver.Add(fraction[(i,j)] <= x[(i,j)])
        
# Maximize value, I can manage delay
objective = solver.Objective()
for i in data['items']:
    for j in data['bags']:
        objective.SetCoefficient(x[(i,j)], data['values'][i])
        #objective.SetCoefficient(fraction[(i,j)], data['values'][i])
objective.SetMaximization()

solv = solver.Solve()
print(solv)

for j in data['bags']:
    bag_value = 0
    bag_weight = 0
    bag_volume= 0
    bag_rad = 0
    print('\n','Bag', j+1 , '\n')
    for i in data['items']:
        if (x[i,j].solution_value()>0) & (fraction[i,j].solution_value() > 0):
            print('Item:', i , 
                  'value',data['values'][i],
                  'Weight', data['weights'][i], 
                  'Volume',data['volume'][i],
                  'Radiation', data['price'][i],
                  'fraction',fraction[i,j].solution_value()
                 )
            bag_value += data['values'][i]
            bag_weight += data['weights'][i]
            bag_volume += data['volume'][i]
            bag_rad += data['price'][i]
    print('Packed Knapsack Value: ', bag_value)
    print('Packed Knapsack Weight: ', bag_weight)
    print('Packed Knapsack Volume: ', bag_volume)
    print('Pack Knapsack Radiation: ', bag_rad)

W0000 00:00:1689553780.388593       1 linear_solver.cc:644] Support for GLPK_LP not linked in, or the license was not found.


AttributeError: 'NoneType' object has no attribute 'IntVar'

In [206]:
to_process.volumen.sum()

124.3445

In [254]:
df.groupby(["folio"],as_index=False).agg({"volumen":"sum","peso":"sum","cod_prod":"size"}).sort_values(by="volumen",ascending = False)



,folio,volumen,peso,cod_prod
350,C23ORI06,243.5261,55619.5160,24
136,C23GRO03,208.5056,47921.7233,23
212,C23MIX02,152.1778,21060.3730,13
423,C23SLP04,151.4239,38060.7741,19
176,C23MER02,136.0534,25244.8430,27
...,...,...,...,...
676,PBP39275,0.0060,27.6000,1
678,PBP39279,0.0040,18.4000,1
677,PBP39277,0.0040,18.4000,1
675,PBP39270,0.0040,18.4000,1


In [258]:
df

,folio,fecha,cod_cte,razon_social,numdpc,sucursal,pobmunedo,cod_prod,descripcion_completa,cantidad_autorizada,peso,volumen,grupo,verificado,nombre,importe,iva,ieps,total,dia
0,PBP38951,2023-01-02 16:38:00,AS17,PRODUCTOS DE TRIGO ...,BP2412,SAHUAYO MORELIA,"MORELIA, MICH",4673,GALLETAS ANIMALITOS CAJA 6/800 GR,150,805.350,3.9450,S8573,1,MICHOACAN- MORELIA,150.0000,0.0,12.0000,162.0000,2023-01-02
1,PBP38951,2023-01-02 16:38:00,AS17,PRODUCTOS DE TRIGO ...,BP2412,SAHUAYO MORELIA,"MORELIA, MICH",3801,GALLETAS ANIMALITOS GRL 1/5 KG,80,403.520,2.0800,S8573,1,MICHOACAN- MORELIA,80.0000,0.0,6.4000,86.4000,2023-01-02
2,PBP38951,2023-01-02 16:38:00,AS17,PRODUCTOS DE TRIGO ...,BP2412,SAHUAYO MORELIA,"MORELIA, MICH",4914,GALLETAS MARIA MARIBEL 20/160 GR,100,359.900,1.6200,S8573,1,MICHOACAN- MORELIA,0.0100,0.0,0.0008,0.0108,2023-01-02
3,PBP38951,2023-01-02 16:38:00,AS17,PRODUCTOS DE TRIGO ...,BP2412,SAHUAYO MORELIA,"MORELIA, MICH",4672,GALLETAS MINI BETUNADAS CAJA 24/140 GR,60,228.780,1.1340,S8573,1,MICHOACAN- MORELIA,1.0000,1.0,1.0000,3.0000,2023-01-02
4,PBP38951,2023-01-02 16:38:00,AS17,PRODUCTOS DE TRIGO ...,BP2412,SAHUAYO MORELIA,"MORELIA, MICH",3702,GALLETAS DORADITA 24/115 GR,100,312.200,1.5000,S8573,1,MICHOACAN- MORELIA,100.0000,0.0,8.0000,108.0000,2023-01-02
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8376,PBP41078,2023-05-04 16:42:00,AS17,PRODUCTOS DE TRIGO ...,BP2205,CENTRAL DE VIVERES VER,"CORDOBA, VER",4914,GALLETAS MARIA MARIBEL 20/160 GR,804,2893.596,13.0248,S9271,1,VERACRUZ,0.0804,0.0,0.0064,0.0868,2023-05-04
8377,PBP41086,2023-05-08 16:18:00,AS17,PRODUCTOS DE TRIGO ...,BP2593,DISTRIBUIDORA OCOTEPEC,"OCOTEPEC,",4914,GALLETAS MARIA MARIBEL 20/160 GR,1500,5398.500,24.3000,S9283,1,NaN,0.1500,0.0,0.0120,0.1620,2023-05-08
8378,PBP41087,2023-05-08 16:18:00,AS17,PRODUCTOS DE TRIGO ...,BP2524,DUERO PUEBLA,"HUEJOTZINGO PUE,",4914,GALLETAS MARIA MARIBEL 20/160 GR,200,719.800,3.2400,S9283,1,PUEBLA- TLAXCALA,0.0200,0.0,0.0016,0.0216,2023-05-08
8379,PBP41096,2023-05-09 15:10:00,AS17,PRODUCTOS DE TRIGO ...,BP2236,ABARROTERA MINA,"COLIMA, COL",3710,GALLETAS TAPATIAS 24/130 PZ,105,366.660,1.5750,S9284,1,NaN,105.0000,0.0,8.4000,113.4000,2023-05-09


In [33]:
### Clean code

In [9]:
import pandas as pd

import ortools
from ortools.linear_solver import pywraplp
import numpy as np
from datetime import datetime
import random
import string

P_WEIGHT = 0.8
P_VOLUMEN = 0.8
W_DELAY = 1 # Castigo para los que paquetes que van quedando viejos

def optimize(data,bag):
    solver = pywraplp.Solver.CreateSolver('SCIP')

    data.update(bag) # se lo apendeo al diccionario
    
    #Prepare solver matrix
    x = {}
    for i in data['items']:
        for j in data['bags']:
            x[(i,j)] = solver.IntVar(0,1,'x_%i_%i' % (i, j))

    #Constraint for an item being placed in 1 knapsack
    for i in data['items']:
        solver.Add(sum(x[i,j] for j in data['bags'])<=1)

    #Knapsack max Constraint
    for j in data['bags']:
        solver.Add(sum(x[(i,j)]*data['weights'][i] for i in data['items']) <= data['bag_weight'][j])
        solver.Add(sum(x[(i,j)]*data['volume'][i] for i in data['items']) <= data['bag_volume'][j])
        solver.Add(sum(x[(i,j)]*data['price'][i] for i in data['items'])*0.1 >= data['bag_price'][j])

    #Knapsack min Constraint
    for j in data['bags']:
        solver.Add(
            (sum(x[(i,j)]*data['volume'][i] for i in data['items']) >= data['bag_volume'][j]*0.8) 
            or
            sum(x[(i,j)]*data['weights'][i] for i in data['items']) >= (data['bag_weight'][j]*0.8)
        )

    # Maximize value, I can manage delay
    objective = solver.Objective()
    for i in data['items']:
        for j in data['bags']:
            objective.SetCoefficient(x[(i,j)], data['values'][i])
    objective.SetMaximization()

    solv = solver.Solve()
    
    # Restore information
    l_ = [[]]

    if solv == pywraplp.Solver.OPTIMAL:
        for j in data['bags']:
            for i in data['items']:
                if x[i,j].solution_value()>0:
                    l_[j] += [i]
                    
    return l_[0] # This is one bag problem

def generate_item_data(raw_data):
    data = {}
    data['values'] = [(datetime.today() - pd.to_datetime(x["fecha"])).days*W_DELAY for x in raw_data]
    data['weights'] = [x["peso"] for x in raw_data]
    data['volume'] = [x["volumen"] for x in raw_data]
    data['price'] = [x["total"] for x in raw_data]
                       
    data['items'] = list(range(len(raw_data)))
    data['num_items'] = len(raw_data)
    
    return data
    
def get_available_truck(metadata,poblacion,volumen,weight,n):
    
    metadata = metadata[metadata["POBLACION"] == poblacion]
    
    posibles_camiones = metadata[(metadata.capacidad_peso*P_WEIGHT <= weight) | 
                                (metadata.capacidad_volumen*P_VOLUMEN <= volumen)]
    # Si no encuentro soluciones. Devuelvo un dataframes con los camiones posibles, si estoy pidiendo el cambion 
    # numero n pero la cantidad de camiones N < n entonces aborto
    if(posibles_camiones.shape[0] <= n): 
        return None
    
    camion = posibles_camiones.sort_values(by="capacidad_peso",ascending = False).to_dict("records")[n] # me traigo el camion pero indexado por iteracion n
    #price = tarifas[(tarifas.POBLACION == poblacion) & (tarifas.TIPO_VEHICULO == camion["TIPO_VEHICULO"])].to_dict("records")[0]
    
    bag = {}
    bag['bag_weight'] = [camion["capacidad_peso"]]
    bag['bag_volume'] = [camion["capacidad_volumen"]] 
    bag['bag_price'] = [camion["TARIFA"]]
    bag["name"] = camion["TIPO_VEHICULO"]
    #I wanted to change the values at a later data
    bag['bags'] = list(range(1))
    
    return bag
    
def log(x,first = False,printer=False,file_name="log.txt"):
    
    x = str(x)
    
    if(first):
        with open(file_name, 'w') as f:
            f.write(x+"\n")
    else:
        with open(file_name, 'a') as f:
            f.write(x+"\n")
            
    if(printer):
        print(x)
        
def rand_name(n):
    return ''.join(random.choices(string.ascii_lowercase + string.digits, k=n))

In [10]:
meta_vehiculos = pd.read_excel("info.xlsx",sheet_name="vehiculos")
tarifario_vehiculos = pd.read_excel("info.xlsx",sheet_name="tarifario").drop_duplicates(subset=["POBLACION","TIPO_VEHICULO"])
tarifario_vehiculos["POBLACION"] = tarifario_vehiculos["POBLACION"].str.strip()
meta_vehiculos_tarifario = pd.merge(meta_vehiculos,tarifario_vehiculos,on="TIPO_VEHICULO")

log("",first=True)
#important_cols = ['folio','pobmunedo', 'cod_prod','cantidad_autorizada','peso', 'volumen', 'total', 'dia']

df = pd.read_excel("info.xlsx",sheet_name="detalle")
df["fecha"] = pd.to_datetime(df["fecha"],infer_datetime_format=True)
df["dia"] = pd.to_datetime(df["fecha"].dt.date,infer_datetime_format=True)
df["pobmunedo"] = df["pobmunedo"].str.strip()


ready = []
df_pending = pd.DataFrame([])

#poblacion = "TORREON, COAH"

for poblacion in df.pobmunedo.unique():
    log(f"********** {poblacion} ***********")
    pending = []
    df_poblacion = df.loc[(df.pobmunedo == poblacion)]

    ######### Esto solo para emular. En el allways on la fecha es fija ######### 
    fechas = df_poblacion.dia.astype("str").sort_values().drop_duplicates().values

    for FECHA in fechas:
        camion_iter = 0
        to_process = df_poblacion[df_poblacion.dia == FECHA]
        pending = to_process.to_dict('records') + pending
        log(f"incoming: {to_process.shape[0]}  Pending: {len(pending) - to_process.shape[0]}")
        ############################################################################ 

        log("------ Iterate through best combination ------")
        for i in range(50): # itero 50 veces por poner un numero grande
            data_to_optimize = generate_item_data(pending) # parseo la data para tenerla en el formato encesario para el opti

            total_volume = sum(data_to_optimize["volume"])
            total_weight = sum(data_to_optimize["weights"])
            log(f"volume, weight to optimize: {total_volume},{total_weight}")

            bag = get_available_truck(meta_vehiculos_tarifario,poblacion,total_volume, # me fijo que camion tengo disponible
                                      total_weight,camion_iter)

            if(bag == None): # si con el peso y volumen no se cumplen las condiciones minimas, termino de loopear
                log("No possible bag")
                break # Ya no hay mas camiones para probar

            items_bag = optimize(data_to_optimize,bag) # optimizo en base al camion disponible
            log(f"Trying with bag: {bag['name']} and optimized: {len(items_bag)}")

            if(len(items_bag) == 0):
                camion_iter += 1 # Si no encuentra solucion pruebo con el siguiente camion para que no se queda iterando infinitamente

            # Updateo los items listos y los pendientes
            ready_iter = list(np.array(pending)[items_bag])
            bag_id = rand_name(4) # asigno un nombre al camion ejemplo: TRAILER_randName
            for i in range(len(ready_iter)): ready_iter[i].update({"camion":f"{bag['name']}_{bag_id}","t_asign":FECHA})
            pending = list(np.array(pending)[[x for x in range(len(pending)) if x not in items_bag]]) # los que me quedaron rezagados, del total que no estna en ready

            ready += ready_iter # save interation

    df_pending = pd.concat([pd.DataFrame(pending),df_pending],axis = 0)

df_ready = pd.DataFrame(ready)

In [11]:
df_ready["t_asign"] = pd.to_datetime(df_ready["t_asign"])
df_ready["delay"] = (df_ready["t_asign"] - df_ready["dia"]).dt.days

df_ready.groupby("pobmunedo").size().mean()

78.33695652173913

In [13]:
df_ready.delay.value_counts().head(20)

0     4295
4      136
22     117
5      114
34     113
9      106
14     101
27     100
32      95
13      91
11      91
8       91
17      86
19      80
18      75
1       72
3       71
6       66
21      65
46      59
Name: delay, dtype: int64

In [96]:
df_ready.delay.value_counts().head(10)

0     4295
4      136
22     117
5      114
34     113
9      106
14     101
27     100
32      95
13      91
Name: delay, dtype: int64

In [59]:
pending_summary = df_pending[["pobmunedo","volumen","peso","total"]].groupby("pobmunedo",as_index=False).sum()
print(pending_summary.shape)
meta_min = meta_vehiculos_tarifario[["capacidad_volumen","capacidad_peso","POBLACION"]].groupby("POBLACION",as_index=False).min()
meta_min = meta_min.rename(columns = {"POBLACION":"pobmunedo"})
pending_summary = pd.merge(pending_summary,meta_min,on="pobmunedo")
print(pending_summary.shape)

explained = pending_summary.loc[(pending_summary.volumen < pending_summary.capacidad_volumen*0.8) & 
                    (pending_summary.peso < pending_summary.capacidad_peso*0.8)]

pending_summary.loc[~pending_summary.pobmunedo.isin(explained.pobmunedo.values)]


(137, 4)
(125, 6)


,pobmunedo,volumen,peso,total,capacidad_volumen,capacidad_peso
3,"ACAYUCAN, VER",100.5255,9260.660,6.339167e+05,84,25000
6,"ALTOTONGA, VER",46.1000,9399.600,1.714835e+05,40,15000
13,"CAMPECHE, CAMP",151.8420,13466.395,1.046703e+06,84,25000
19,"CHIHUITAN, OAX",39.9921,22056.500,1.153546e+06,84,25000
20,"CHILPANCINGO, GRO",103.8234,12039.727,7.692594e+05,40,15000
21,"COATEPEC, VER",34.3680,6822.500,1.259669e+05,40,15000
26,"CORDOBA, VER",44.0990,9032.342,1.101361e+05,40,15000
39,"GUADALAJARA, JAL",19.9449,3996.768,9.691222e+02,21,5000
45,"JILOTLAN, JAL",26.5300,6044.724,1.692360e+03,21,5000
49,"LEON, GTO",27.1115,5952.895,3.571080e+02,21,5000


In [62]:
df_pending.loc[df_pending.pobmunedo == "ACAYUCAN, VER"]

,folio,fecha,cod_cte,razon_social,numdpc,sucursal,pobmunedo,cod_prod,descripcion_completa,cantidad_autorizada,peso,volumen,grupo,verificado,nombre,importe,iva,ieps,total,dia
0,C23ACA10,2023-03-23 17:42:00,110101,DICONSA ...,WL628,3001 CENTRAL ACAYUCAN,"ACAYUCAN, VER",4826,JABON DE LAVANDERIA FLORAL ROSA 20/350 GR,300,2265.00,2.8413,S9030,1,16 GOLFO - SURESTE,104388.0,16702.08,0.0,121090.08,2023-03-23
1,C23ACA05,2023-02-14 09:08:00,110101,DICONSA ...,WL628,3001 CENTRAL ACAYUCAN,"ACAYUCAN, VER",73113900SCA,HIG REGIO Rinde+ 16x4s 400HD 16/4 PZ,1070,6995.66,97.6842,S8898,1,16 GOLFO - SURESTE,442091.9,70734.70,0.0,512826.60,2023-02-14


In [32]:
df_ready[["pobmunedo","volumen","peso","total","camion"]].groupby(["pobmunedo","camion"]).sum()

volumen        peso         total
pobmunedo           camion                                           
ACAPONETA, NAY      TORTON_ed3j     36.8124  10126.2425  5.392489e+05
                    TORTON_f7lb     37.4242  11755.6716  6.915995e+05
                    TORTON_nebt     36.6313  10448.6145  5.673321e+05
AGUASCALIENTES, AGS TORTON_9mrt     32.8115   6791.2450  9.859296e+04
                    TORTON_lkdv     37.3513   7331.2190  1.981954e+05
...                                     ...         ...           ...
ZAPOPAN, JAL        CAMIONETA_w16v  20.7360   4700.9280  1.604635e+05
                    CAMIONETA_wgb4  17.9341   3837.1810  7.797116e+04
                    TORTON_7z1r     36.7499   9252.0280  5.057840e+05
                    TORTON_vlnd     37.7389   6070.2488  3.214711e+05
                    TRAILER_11da    79.8486  22061.9100  1.150534e+06

[208 rows x 3 columns]

In [50]:
pending_summary.shape

(137, 4)

In [47]:
get_available_truck(meta_vehiculos_tarifario,"ZAMORA, MICH",45.591,10313.382000000001,1)

    TIPO_VEHICULO  capacidad_volumen  capacidad_peso     POBLACION  TARIFA
395        TORTON                 40           15000  ZAMORA, MICH    6800
